# Week 3. Peer-graded Assignment. Part II
## Segmenting and Clustering Neighborhoods in Toronto.

In [1]:
!conda install -y lxml

import pandas as pd

Solving environment: done

# All requested packages already installed.



### Step 1
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_table_list = pd.read_html(wikipedia_url)

raw_table = raw_table_list[0]

The dataframe will consist of **three columns**: PostalCode, Borough, and Neighborhood

In [3]:
raw_table.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Step 2
Only process the cells that have an assigned borough. **Ignore cells with a borough that is Not assigned**.

In [4]:
raw_table.drop(raw_table[raw_table.Borough == 'Not assigned'].index, inplace = True)
raw_table.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Step 3
**More than one neighborhood can exist in one postal code area**. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be **combined into one row with the neighborhoods separated with a comma** as shown in row 11 in the above table.

In [5]:
grouped_table = raw_table.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [6]:
grouped_table['Neighborhood'] = grouped_table['Neighborhood'].apply(lambda x: x.replace('/', ','))
grouped_table

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


### Step 4
If a cell **has a borough but a *Not assigned* neighborhood**, then the **neighborhood will be the same as the borough**.

In [7]:
for index, row in grouped_table.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

### Step 5
In the last cell of your notebook, **use the *.shape* method to print the number of rows** of your dataframe

In [8]:
grouped_table.shape

(103, 3)

In [9]:
pc_file_url = 'http://cocl.us/Geospatial_data'
postal_codes = pd.read_csv(pc_file_url)

postal_codes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [16]:
grouped_table.rename(columns={'Postal code': 'Postal Code'}, inplace=True)

In [17]:
final_table = pd.merge(grouped_table, postal_codes, on='Postal Code')

In [18]:
final_table

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
